# Math Task Processor for Kaggle

Этот ноутбук обрабатывает математические задачи из CSV файла с помощью модели Qwen 2.5 7B через HuggingFace API.

## Инструкции по запуску:

1. **Загрузите файлы в Kaggle:**
   - `test_private.csv` - файл с задачами
   - `system.txt` - системный промпт

2. **Добавьте HuggingFace токен:**
   - В Kaggle Secrets добавьте `HUGGINGFACE_API_TOKEN`
   - Или измените переменную `HF_TOKEN` в коде ниже

3. **Запустите все ячейки**

In [ ]:
# Установка зависимостей
!pip install transformers torch accelerate bitsandbytes pandas python-dotenv

In [ ]:
import pandas as pd
import json
import os
import time
from datetime import datetime
from pathlib import Path
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import gc
from typing import Dict, Any, List

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA device: {torch.cuda.get_device_name()}")
    print(f"CUDA memory: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB")

In [ ]:
# Конфигурация
MODEL_NAME = "Qwen/Qwen2.5-7B-Instruct"
CSV_FILE = "/kaggle/input/aic-2025-math/test_private.csv"  # Путь к загруженному CSV файлу
SYSTEM_PROMPT_FILE = "/kaggle/input/aic-2025-math/system.txt"  # Путь к системному промпту
OUTPUT_DIR = "/kaggle/working/outputs"

# HuggingFace токен (используйте Kaggle Secrets или укажите напрямую)
try:
    from kaggle_secrets import UserSecretsClient
    user_secrets = UserSecretsClient()
    HF_TOKEN = user_secrets.get_secret("HUGGINGFACE_API_TOKEN")
    print("✅ HuggingFace token loaded from Kaggle Secrets")
except:
    HF_TOKEN = "your_huggingface_token_here"  # Замените на ваш токен
    print("⚠️ Using hardcoded token - consider using Kaggle Secrets")

# Создаем выходную директорию
os.makedirs(OUTPUT_DIR, exist_ok=True)
print(f"Output directory: {OUTPUT_DIR}")

In [ ]:
# Загрузка системного промпта
def load_system_prompt(file_path: str) -> str:
    """Загружает системный промпт из файла."""
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            return f.read().strip()
    except FileNotFoundError:
        print(f"⚠️ System prompt file not found: {file_path}")
        return """ВЫ — СТРОГИЙ ПАРСЕР МАТЕМАТИЧЕСКИХ ЗАДАЧ.
Ваша единственная задача: преобразовать входной текст задачи в валидный JSON строго по схеме MathIR-JSON. 
НИКОГДА не решайте задачу и не добавляйте отсутствующие факты.
Возвращайте ТОЛЬКО JSON. Никакого текста, комментариев, Markdown и пояснений."""

system_prompt = load_system_prompt(SYSTEM_PROMPT_FILE)
print(f"System prompt loaded: {len(system_prompt)} characters")
print(f"First 100 chars: {system_prompt[:100]}...")

In [ ]:
# Загрузка задач из CSV
def load_tasks(csv_path: str) -> pd.DataFrame:
    """Загружает задачи из CSV файла."""
    try:
        df = pd.read_csv(csv_path)
        print(f"✅ Loaded {len(df)} tasks from {csv_path}")
        return df
    except FileNotFoundError:
        print(f"❌ CSV file not found: {csv_path}")
        # Создаем тестовые данные если файл не найден
        test_data = {
            'task': [
                'Вычислить предел: lim(x→0) sin(x)/x',
                'Вычислить интеграл: ∫₀^π sin(x) dx',
                'Найти производную: d/dx(x²+2x+1)'
            ]
        }
        df = pd.DataFrame(test_data)
        print(f"⚠️ Using test data: {len(df)} tasks")
        return df

df = load_tasks(CSV_FILE)
print(f"Sample task: {df.iloc[0]['task'][:100]}...")

In [ ]:
# Настройка модели с квантизацией для экономии памяти
def setup_model():
    """Настраивает модель Qwen 2.5 7B с квантизацией."""
    print("🔄 Loading Qwen 2.5 7B model...")
    
    # Конфигурация квантизации для экономии памяти
    quantization_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_compute_dtype=torch.float16,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type="nf4"
    )
    
    # Загрузка токенизатора
    tokenizer = AutoTokenizer.from_pretrained(
        MODEL_NAME,
        token=HF_TOKEN,
        trust_remote_code=True
    )
    
    # Загрузка модели с квантизацией
    model = AutoModelForCausalLM.from_pretrained(
        MODEL_NAME,
        token=HF_TOKEN,
        quantization_config=quantization_config,
        device_map="auto",
        trust_remote_code=True,
        torch_dtype=torch.float16
    )
    
    print("✅ Model loaded successfully")
    return tokenizer, model

tokenizer, model = setup_model()

In [ ]:
# Функция для генерации ответов
def generate_response(task: str, tokenizer, model, max_length: int = 2048) -> str:
    """Генерирует ответ модели для задачи."""
    
    # Создаем полный промпт
    full_prompt = f"{system_prompt}\n\nЗадача: {task}"
    
    # Форматируем для Qwen
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": f"Задача: {task}"}
    ]
    
    # Применяем chat template
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )
    
    # Токенизация
    inputs = tokenizer(
        text,
        return_tensors="pt",
        truncation=True,
        max_length=4096
    ).to(model.device)
    
    # Генерация
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_length,
            temperature=0.1,
            top_p=0.9,
            do_sample=True,
            pad_token_id=tokenizer.eos_token_id
        )
    
    # Декодирование ответа
    response = tokenizer.decode(
        outputs[0][inputs['input_ids'].shape[1]:],
        skip_special_tokens=True
    )
    
    return response.strip()

# Тест функции генерации
test_task = "Вычислить предел: lim(x→0) sin(x)/x"
print("🧪 Testing generation function...")
start_time = time.time()
test_response = generate_response(test_task, tokenizer, model)
test_time = time.time() - start_time
print(f"✅ Test completed in {test_time:.2f}s")
print(f"Response length: {len(test_response)} characters")
print(f"First 200 chars: {test_response[:200]}...")

In [ ]:
# Функция для обработки всех задач
def process_all_tasks(df: pd.DataFrame, tokenizer, model) -> List[Dict[str, Any]]:
    """Обрабатывает все задачи из DataFrame."""
    results = []
    total_tasks = len(df)
    
    print(f"🚀 Starting processing of {total_tasks} tasks...")
    
    for index, row in df.iterrows():
        task_id = f"task_{index+1:06d}"
        task = row['task']
        
        print(f"\n📝 Processing {task_id} ({index+1}/{total_tasks})")
        print(f"Task: {task[:100]}...")
        
        try:
            start_time = time.time()
            response = generate_response(task, tokenizer, model)
            latency = time.time() - start_time
            
            # Создаем результат
            result = {
                "task_id": task_id,
                "task": task,
                "response": response,
                "latency": latency,
                "timestamp": datetime.now().isoformat(),
                "status": "success"
            }
            
            # Пытаемся извлечь JSON из ответа
            response_text = response.strip()
            if response_text.startswith('{') and response_text.endswith('}'):
                try:
                    parsed_json = json.loads(response_text)
                    result["parsed_json"] = parsed_json
                    print(f"✅ Valid JSON extracted")
                except json.JSONDecodeError:
                    print(f"⚠️ Response looks like JSON but failed to parse")
            
            results.append(result)
            print(f"✅ {task_id} completed in {latency:.2f}s")
            
            # Сохраняем промежуточные результаты каждые 10 задач
            if (index + 1) % 10 == 0:
                save_intermediate_results(results, index + 1)
                
            # Очистка памяти
            if (index + 1) % 50 == 0:
                gc.collect()
                torch.cuda.empty_cache()
                print(f"🧹 Memory cleanup after {index + 1} tasks")
                
        except Exception as e:
            print(f"❌ {task_id} failed: {str(e)}")
            result = {
                "task_id": task_id,
                "task": task,
                "error": str(e),
                "timestamp": datetime.now().isoformat(),
                "status": "error"
            }
            results.append(result)
    
    return results

def save_intermediate_results(results: List[Dict[str, Any]], count: int):
    """Сохраняет промежуточные результаты."""
    filename = f"{OUTPUT_DIR}/intermediate_results_{count:03d}.json"
    with open(filename, 'w', encoding='utf-8') as f:
        json.dump(results, f, ensure_ascii=False, indent=2)
    print(f"💾 Intermediate results saved: {filename}")

In [ ]:
# Обработка всех задач
print("🚀 Starting full processing...")
start_time = time.time()

results = process_all_tasks(df, tokenizer, model)

total_time = time.time() - start_time
print(f"\n🎉 Processing completed in {total_time:.2f}s ({total_time/60:.1f} minutes)")

In [ ]:
# Сохранение финальных результатов
def save_final_results(results: List[Dict[str, Any]]):
    """Сохраняет финальные результаты."""
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    
    # Сохраняем индивидуальные файлы
    individual_dir = f"{OUTPUT_DIR}/individual"
    os.makedirs(individual_dir, exist_ok=True)
    
    for result in results:
        task_id = result["task_id"]
        with open(f"{individual_dir}/{task_id}.json", 'w', encoding='utf-8') as f:
            json.dump(result, f, ensure_ascii=False, indent=2)
    
    # Сохраняем все результаты
    all_results_file = f"{OUTPUT_DIR}/all_results_{timestamp}.json"
    with open(all_results_file, 'w', encoding='utf-8') as f:
        json.dump(results, f, ensure_ascii=False, indent=2)
    
    # Создаем отчет
    successful = sum(1 for r in results if r["status"] == "success")
    failed = len(results) - successful
    total_latency = sum(r.get("latency", 0) for r in results if "latency" in r)
    avg_latency = total_latency / successful if successful > 0 else 0
    
    report = {
        "timestamp": datetime.now().isoformat(),
        "total_tasks": len(results),
        "successful": successful,
        "failed": failed,
        "success_rate": successful / len(results) if results else 0,
        "total_latency": total_latency,
        "average_latency": avg_latency,
        "model": MODEL_NAME
    }
    
    report_file = f"{OUTPUT_DIR}/report_{timestamp}.json"
    with open(report_file, 'w', encoding='utf-8') as f:
        json.dump(report, f, ensure_ascii=False, indent=2)
    
    print(f"\n📊 Final Report:")
    print(f"Total tasks: {report['total_tasks']}")
    print(f"Successful: {report['successful']}")
    print(f"Failed: {report['failed']}")
    print(f"Success rate: {report['success_rate']:.2%}")
    print(f"Average latency: {report['average_latency']:.2f}s")
    print(f"\n💾 Files saved:")
    print(f"- Individual results: {individual_dir}/")
    print(f"- All results: {all_results_file}")
    print(f"- Report: {report_file}")
    
    return report

# Сохраняем результаты
final_report = save_final_results(results)

In [ ]:
# Анализ результатов
def analyze_results(results: List[Dict[str, Any]]):
    """Анализирует результаты обработки."""
    print("📈 Results Analysis:")
    print("=" * 50)
    
    # Статистика по времени
    successful_results = [r for r in results if r["status"] == "success" and "latency" in r]
    if successful_results:
        latencies = [r["latency"] for r in successful_results]
        print(f"⏱️ Timing Statistics:")
        print(f"  Min latency: {min(latencies):.2f}s")
        print(f"  Max latency: {max(latencies):.2f}s")
        print(f"  Avg latency: {sum(latencies)/len(latencies):.2f}s")
    
    # Статистика по JSON
    json_results = [r for r in results if "parsed_json" in r]
    print(f"\n📋 JSON Statistics:")
    print(f"  Valid JSON responses: {len(json_results)}/{len(successful_results)}")
    print(f"  JSON success rate: {len(json_results)/len(successful_results)*100:.1f}%")
    
    # Примеры ошибок
    error_results = [r for r in results if r["status"] == "error"]
    if error_results:
        print(f"\n❌ Error Examples:")
        for i, error_result in enumerate(error_results[:3]):
            print(f"  {i+1}. {error_result['task_id']}: {error_result['error'][:100]}...")
    
    # Примеры успешных ответов
    if json_results:
        print(f"\n✅ JSON Response Examples:")
        for i, json_result in enumerate(json_results[:2]):
            print(f"  {i+1}. {json_result['task_id']}:")
            print(f"     Task: {json_result['task'][:80]}...")
            print(f"     JSON keys: {list(json_result['parsed_json'].keys())}")

analyze_results(results)

In [ ]:
# Создание архива для скачивания
import zipfile

def create_download_archive():
    """Создает ZIP архив с результатами для скачивания."""
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    archive_name = f"{OUTPUT_DIR}/math_results_{timestamp}.zip"
    
    with zipfile.ZipFile(archive_name, 'w', zipfile.ZIP_DEFLATED) as zipf:
        # Добавляем все файлы из output директории
        for root, dirs, files in os.walk(OUTPUT_DIR):
            for file in files:
                if file.endswith('.json'):
                    file_path = os.path.join(root, file)
                    arcname = os.path.relpath(file_path, OUTPUT_DIR)
                    zipf.write(file_path, arcname)
    
    print(f"📦 Archive created: {archive_name}")
    print(f"📁 Archive size: {os.path.getsize(archive_name) / 1024 / 1024:.2f} MB")
    return archive_name

archive_path = create_download_archive()
print(f"\n🎯 Download your results: {archive_path}")

In [ ]:
# Финальная сводка
print("\n" + "="*60)
print("🎉 PROCESSING COMPLETED!")
print("="*60)
print(f"📊 Summary:")
print(f"  • Total tasks processed: {len(results)}")
print(f"  • Successful: {sum(1 for r in results if r['status'] == 'success')}")
print(f"  • Failed: {sum(1 for r in results if r['status'] == 'error')}")
print(f"  • Success rate: {sum(1 for r in results if r['status'] == 'success')/len(results)*100:.1f}%")
print(f"  • Total time: {sum(r.get('latency', 0) for r in results)/60:.1f} minutes")
print(f"\n📁 Output files:")
print(f"  • Individual JSON files: {OUTPUT_DIR}/individual/")
print(f"  • Combined results: {OUTPUT_DIR}/all_results_*.json")
print(f"  • Processing report: {OUTPUT_DIR}/report_*.json")
print(f"  • Download archive: {archive_path}")
print(f"\n🔧 Model used: {MODEL_NAME}")
print(f"📝 System prompt: {len(system_prompt)} characters")
print("\n✅ All tasks completed successfully!")